In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
pd.set_option('display.max_columns', 50)

In [3]:
test=pd.read_csv("./dataset_labeled/nsl_testing_labeled.csv")
tra=pd.read_csv("./dataset_labeled/nsl_training_labeled.csv")

In [4]:
test.head()

,Duration,ProtocolType,Service,Flag,SourceBytes,DestinationBytes,Land,WrongFragment,Urgent,Hot,FailedLogins,LoggedIn,NumberOfCompromised,RootShell,SuAttempted,Root,NumberOfFileCreations,NumberOfShellPrompts,NumberOfAccessFiles,NumberOfOutbound,IsHotLogin,IsGuestLogin,Count,SrvCount,SerrorRate,SrvSerrorRate,RerrorRate,SrvRerrorRate,SameSrvRate,DiffSrvRate,SrvDiffHostRate,DstHostCount,DstHostSrvCount,DstHostSameSrvRate,DstHostDiffSrvRate,DstHostSameSrcPortRate,DstHostSrvDiffHostRate,DstHostSerrorRate,DstHostSrvSerrorRate,DstHostRerrorRate,DstHostSrvRerrorRate,TypeOfAttack,TimesApparead,Group,IndexAttack,IndexByGroup,IndexBinary
0,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,229,10,0.0,0.00,1.0,1.0,0.04,0.06,0.00,255,10,0.04,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21,dos,14,1,1
1,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,1,0.0,0.00,1.0,1.0,0.01,0.06,0.00,255,1,0.00,0.06,0.00,0.00,0.0,0.0,1.00,1.00,neptune,21,dos,14,1,1
2,2,tcp,ftp_data,SF,12983,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.0,0.00,0.0,0.0,1.00,0.00,0.00,134,86,0.61,0.04,0.61,0.02,0.0,0.0,0.00,0.00,normal,21,normal,16,2,0
3,0,icmp,eco_i,SF,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,65,0.0,0.00,0.0,0.0,1.00,0.00,1.00,3,57,1.00,0.00,1.00,0.28,0.0,0.0,0.00,0.00,saint,15,probe,24,3,1
4,1,tcp,telnet,RSTO,0,15,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8,0.0,0.12,1.0,0.5,1.00,0.00,0.75,29,86,0.31,0.17,0.03,0.02,0.0,0.0,0.83,0.71,mscan,11,probe,11,3,1


In [5]:
#definiendo funcion para calcular la informacion y entropia
s_informacion=None
s_entropia=None
#serie tiene cada valor con su cantidad
def calculo_informacion_entropia(serie_valores_unicos,probabilidad):
    s_informacion=[]
    s_entropia=[]
    for indice in list(range(0,len(serie_valores_unicos))):
        info=-1*math.log(probabilidad.iloc[indice],2)
        entropia=probabilidad.iloc[indice]*info
        s_informacion.append(info)
        s_entropia.append(entropia)
    ps_informacion=pd.Series(s_informacion)
    ps_entropia=pd.Series(s_entropia)
    return (ps_informacion,ps_entropia)

In [6]:
#definiendo funcion para calcular la probabilidad. da más de 1 la suma
s_probabilidad=None
def probabilidad(serie):
    s_probabilidad=[]
    e_agrupados = serie.value_counts()
    for indice in list(range(0,len(e_agrupados))):
        proba=e_agrupados.iloc[indice]/e_agrupados.sum()
        s_probabilidad.append(proba)
    ps_probabilidad=pd.Series(s_probabilidad)
    return (ps_probabilidad,e_agrupados)

In [7]:
def entropia_informacion(dataset,columnas=[2,3,4,7,12,14,15,21,22]):
    #cleaning the dataset
    #restando uno al subindice, ya que pandas empieza la numeracion en 0
    #volviendo un numpy array porque en las listas no hay restas
    subindice=np.asarray(columnas)-1
    #regresando a la lista
    subindice=subindice.tolist()
    indice=list(range(0,len(dataset.columns)-6))
    indice.extend(subindice)
    s_indice=pd.Series(indice)
    #eleminando los duplicados en el Series de Pandas
    s_indice=s_indice.drop_duplicates(keep=False)
    for columna in s_indice:
        #serie temporal
        st=dataset.loc[:,dataset.columns[columna]]
        [proba,serie_agrupada]=probabilidad(st)
        [informacion,entropia]=calculo_informacion_entropia(serie_agrupada,proba)
        print("Columna no: %d, etiquetada: %s" %(columna+1,dataset.columns[columna]))
        print("Su Promedio es: %f" %(st.mean()))
        print("Su Desviación Estandar es: %f" % (st.std()))
        print("Su Informacion es: %f" % (informacion.sum()))
        print("Su entropia es: %f" % (entropia.sum()))        

In [8]:
entropia_informacion(test)

Columna no: 1, etiquetada: Duration
Su Promedio es: 218.868784
Su Desviación Estandar es: 1407.207069
Su Informacion es: 8557.617969
Su entropia es: 1.560294
Columna no: 5, etiquetada: SourceBytes
Su Promedio es: 10395.911325
Su Desviación Estandar es: 472796.912693
Su Informacion es: 14849.668090
Su entropia es: 6.277053
Columna no: 6, etiquetada: DestinationBytes
Su Promedio es: 2056.110012
Su Desviación Estandar es: 21219.763847
Su Informacion es: 50219.489746
Su entropia es: 6.529377
Columna no: 8, etiquetada: WrongFragment
Su Promedio es: 0.008428
Su Desviación Estandar es: 0.142602
Su Informacion es: 17.653985
Su entropia es: 0.045463
Columna no: 9, etiquetada: Urgent
Su Promedio es: 0.000710
Su Desviación Estandar es: 0.036474
Su Informacion es: 39.059888
Su entropia es: 0.006185
Columna no: 10, etiquetada: Hot
Su Promedio es: 0.105399
Su Desviación Estandar es: 0.928448
Su Informacion es: 179.049269
Su entropia es: 0.308209
Columna no: 11, etiquetada: FailedLogins
Su Promedio e